In [9]:
## Import packages

import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.neighbors import KNeighborsClassifier

In [10]:
## Import CSVs and break into training and testing tables

train = pd.read_csv('train.csv').convert_dtypes()

train_x = train.drop(columns=['Survived'], axis=1)
train_y = pd.DataFrame(train, columns=['Survived'])

test_x = pd.read_csv('test.csv').convert_dtypes()

In [12]:
## Calculate average age for each Sex and Pclass combination

avg_age = train_x.pivot_table(
    columns=['Sex', 'Pclass'], values='Age', aggfunc='mean').mean()

avg_age = avg_age.reset_index()
avg_age.columns = ['Sex', 'Pclass', 'Value']

print(avg_age)


      Sex  Pclass      Value
0  female       1  34.611765
1  female       2  28.722973
2  female       3  21.750000
3    male       1  41.281386
4    male       2  30.740707
5    male       3  26.507589


In [13]:
## Replace missing ages with average age for Sex and Pclass

train_x = train_x.merge(avg_age, left_on=['Sex', 'Pclass'], right_on=[
                    'Sex', 'Pclass'], how='left')
train_x['Age'].fillna(train_x['Value'], inplace=True)

test_x = test_x.merge(avg_age, left_on=['Sex', 'Pclass'], right_on=[
                  'Sex', 'Pclass'], how='left')
test_x['Age'].fillna(test_x['Value'], inplace=True)


In [ ]:
## Convert male/female text to numbers

train_x.loc[train_x['Sex'] == 'male', 'Sex_as_num'] = 0
train_x.loc[train_x['Sex'] == 'female', 'Sex_as_num'] = 1

test_x.loc[test_x['Sex'] == 'male', 'Sex_as_num'] = 0
test_x.loc[test_x['Sex'] == 'female', 'Sex_as_num'] = 1

In [14]:
## Remove text columns from training and testing dataframes and index

train_x.set_index('PassengerId', inplace=True)
train_x.drop(columns=['Name', 'Ticket', 'Cabin', 'Fare',
           'Embarked', 'Sex', 'Value'], inplace=True)

train_y.set_index(train_x.index, inplace=True)

test_x.set_index('PassengerId', inplace=True)
test_x.drop(columns=['Name', 'Ticket', 'Cabin', 'Fare',
          'Embarked', 'Sex', 'Value'], inplace=True)

In [16]:
## Apply KNN algorithm to training set and use it to predict test set

knn = KNeighborsClassifier(n_neighbors=7)
knn.fit(train_x, train_y)

prediction = knn.predict(test_x)

/opt/homebrew/Caskroom/miniconda/base/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:200: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [18]:
## Format output table for submission

train_x['Survived'] = train_y

submit = test_x.drop(
    columns=['Pclass', 'Age', 'SibSp', 'Parch', 'Sex_as_num']).reset_index()
submit.to_csv('submission.csv', index=False)

In [19]:
## Import answer

test_y = pd.read_csv('answer.csv').convert_dtypes()

In [20]:
test_y.set_index(test_x.index, inplace=True)
test_y.drop(columns=['PassengerId'], inplace=True)

In [21]:
test_x['Survived'] = test_y

In [23]:
train_x.head()

,Pclass,Age,SibSp,Parch,Sex_as_num,Survived
PassengerId,,,,,,
1,3,22.0,1,0,0.0,0
2,1,38.0,1,0,1.0,1
3,3,26.0,0,0,1.0,1
4,1,35.0,1,0,1.0,1
5,3,35.0,0,0,0.0,0
